In [3]:
from q_value_calc_crosslinks import calcQ
from idXML2df_all_hits import readAndProcessIdXMLAllHits
from functions import get_target_id, read_top_down_data, get_datasets, rerank
import re
import os
import numpy as np

In [4]:
i = 1
dataset = get_datasets()

In [5]:
directory = f"../data/{dataset[i]['type']}/"
input_files = [directory + f"{dataset[i]['file']}{dataset[i]['file_ending']}"]
if dataset[i]['comparison'] != None: 
      input_files.append(directory + f"{dataset[i]['file']}_{dataset[i]['comparison']}{dataset[i]['file_ending']}")
print(input_files)

['../data/crosslink_data/AChernev_080219_HeLa_RNA_UV.idXML']


In [6]:
directory += f"{dataset[i]['name']}/"
if not os.path.exists(directory):
    os.makedirs(directory)
for input_file in input_files:
    if dataset[i]['type'] == 'crosslink_data':
        df = readAndProcessIdXMLAllHits(input_file)
        classColName = 'NuXL:isXL'
    elif dataset[i]['type'] == 'top_down_data':
        df = read_top_down_data(input_file)
        df['ScanNr'] = df['Scan']
        df = rerank(df, 'ScanNr', 'Score', 'PSMId')
        classColName = 'ModCount'
    file_ending = dataset[i]['file_ending']
    
    # filter dataframes
    filter_col = 'PSMId'
    filter_val = 1
    mask = df[df[filter_col] == filter_val].index

    # compute q-values
    df.sort_values(['Score', 'Label'], ascending=[False, True], inplace=True)
    df['class-specific_q-val'] = np.NaN
    df['cum_target_id'] = np.NaN
    df.loc[mask,:] = calcQ(df.loc[mask,:], classColName = classColName)
    # compute target IDs
    df.loc[mask,:] = get_target_id(df.loc[mask,:], isXLColName=dataset[i]['group'])
    output_file = f"{dataset[i]['file']}"
    if dataset[i]['comparison'] != None and dataset[i]['comparison'] in input_file: 
        output_file += f"_{dataset[i]['comparison']}"
    output_file += ".pkl"
    df.to_pickle(directory + output_file)

c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


In [7]:
print(directory)

../data/crosslink_data/AChernev_080219/
